For this hands-on, we will be using the [Power Plant dataset](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant) dataset where the goal is to predict the net hourly electrical energy output (EP) of a plant.

In [26]:
from datetime import datetime

import mlflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

pd.set_option("display.max_columns", None)

In [27]:
df = pd.read_csv("../data/power_plants.csv")
df.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


# MLflow Tracking

## Model traning

In [28]:
def train_model(train_df, max_depth=2):
    # Split data
    X = train_df[["AT", "V", "AP", "RH"]]
    y = train_df["PE"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit model
    model = RandomForestRegressor(max_depth=max_depth)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f"Test mse = {mse}, Test RMSE = {rmse}, Random forest max depth = {max_depth}")
    return model, mse, rmse

In [29]:
_ = train_model(df, max_depth=2)

2022/05/11 11:44:54 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b00a5712a5c24f6eba551e07cb863ecd', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Test mse = 37.22579981648007, Test RMSE = 6.1012949294785015, Random forest max depth = 2


- Test with different max depths for the Random forest

In [30]:
for max_depth in range(2, 7, 2):
    _ = train_model(df, max_depth=max_depth)

2022/05/11 11:45:10 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8537e820be164d9aa325b96dfa022f6d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/05/11 11:45:13 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '60d4cea4729b495b8456cb0d131678b7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Test mse = 37.38716474179541, Test RMSE = 6.114504455946975, Random forest max depth = 2


2022/05/11 11:45:18 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e654e0cc032e46469a4ce2b577cac4cb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Test mse = 19.993436971773324, Test RMSE = 4.471402125930224, Random forest max depth = 4
Test mse = 14.590129139161833, Test RMSE = 3.8197027553412886, Random forest max depth = 6


## Experiment tracking

### Some vocabulary:
- **run**: single execution of model training code. Each run can record different informations (model parameters, metrics, tags, artifacts, etc).
- **experiment**: the primary unit of organization and access control for MLflow runs; all MLflow runs belong to an experiment. Experiments let you visualize, search for, and compare runs, as well as download run artifacts and metadata for analysis in other tools.

In [31]:
!ls

mlflow_tracking_hands_on.ipynb mlruns


In [32]:
experiment_name = "ep_prediction_with_random_forest"
mlflow.set_experiment(experiment_name)

In [33]:
!ls

mlflow_tracking_hands_on.ipynb mlruns


### Basic logging
- Log model hyper-parameters, metric and the model itself

In [34]:
def train_model(train_df, max_depth=2):
    with mlflow.start_run():
        # Split data
        X = train_df[["AT", "V", "AP", "RH"]]
        y = train_df["PE"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Fit model
        model = RandomForestRegressor(max_depth=max_depth)
        model.fit(X_train, y_train)
        ## mlflow: log model & its hyper-parameters
        mlflow.log_param("max_depth", max_depth)
        mlflow.sklearn.log_model(model, "model")

        # Evaluate the model
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        ## mlflow: log metrics
        mlflow.log_metrics({"testing_mse": mse, "testing_rmse": rmse})
        print(f"Test mse = {mse}, Test RMSE = {rmse}, Random forest max depth = {max_depth}")

- Run the function with mlflow tracking

In [35]:
for max_depth in range(2, 7, 2):
    _ = train_model(df, max_depth=max_depth)

Test mse = 37.88724076563485, Test RMSE = 6.155261226433436, Random forest max depth = 2
Test mse = 20.13032800426607, Test RMSE = 4.486683408071721, Random forest max depth = 4
Test mse = 14.592045410594258, Test RMSE = 3.8199535874921646, Random forest max depth = 6


### Visualize experiments with MLflow tracking UI

To run the [MLflow Tracking UI](https://www.mlflow.org/docs/latest/tracking.html#tracking-ui), you need to either run the UI with ```mlflow ui``` (needs to be executed from the *notebooks* folder) oor to run an *mlflow server* (will be used in the following section)

### Where mlflow saves the data

#### Some vocabulary:
- **Backend store**: for MLflow entities (runs, parameters, metrics, tags, notes, metadata, etc)
- **Artefact store**: for artifacts (files, models, images, in-memory objects, etc)
- For more information, [check the official documentation](https://www.mlflow.org/docs/latest/tracking.html#where-runs-are-recorded)

#### Without prior configuration
- When no pror configuration is set, MLflow creates an *mlruns* folder where the data will be saved

In [36]:
!ls

mlflow_tracking_hands_on.ipynb mlruns


- MLflow created a new folder *mlruns* where it will store the different run informations

In [37]:
!tree mlruns

mlruns
├── 0
│   └── meta.yaml
└── 1
    ├── 426c882475304ce48a5172fa4906b2fc
    │   ├── artifacts
    │   │   └── model
    │   │       ├── MLmodel
    │   │       ├── conda.yaml
    │   │       ├── model.pkl
    │   │       └── requirements.txt
    │   ├── meta.yaml
    │   ├── metrics
    │   │   ├── testing_mse
    │   │   └── testing_rmse
    │   ├── params
    │   │   └── max_depth
    │   └── tags
    │       ├── mlflow.log-model.history
    │       ├── mlflow.source.name
    │       ├── mlflow.source.type
    │       └── mlflow.user
    ├── 45bef963751543f495c5a7c37eb1dbc3
    │   ├── artifacts
    │   │   └── model
    │   │       ├── MLmodel
    │   │       ├── conda.yaml
    │   │       ├── model.pkl
    │   │       └── requirements.txt
    │   ├── meta.yaml
    │   ├── metrics
    │   │   ├── testing_mse
    │   │   └── testing_rmse
    │   ├── params
    │   │   └── max_depth
    │   └── tags
    │       ├── mlflow.log-model.history
    │       ├── mlflow.source.name
    

#### With prior configuration
- Set the **Backend store** to an sqlite database located in */tmp/mlruns.db* and the **Artefact store**  to a folder located in */tmp/mlruns*. For more informations on the different possibilities available (S3, blobstorage, etc) check [the official documentation](https://www.mlflow.org/docs/latest/tracking.html#where-runs-are-recorded).
- To run the MLflow server, you needd to execute the following command in your terminal
```mlflow server --backend-store-uri sqlite:////tmp/mlruns.db --default-artifact-root /tmp/mlruns```
- Set the tracking uri in the notebook ```mlflow.set_tracking_uri('http://127.0.0.1:5000')```

In [38]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [39]:
# Create the experiment in the new database
experiment_name = "ep_prediction_with_random_forest"
mlflow.set_experiment(experiment_name)

### Loggiong with autolog

- Autollog will log all the model parameters, training metrics, model binary, etc **BUT not the test metrics**, tthey needd to be logged manually

In [40]:
def train_model(train_df, max_depth=2):
    training_timestamp = datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    with mlflow.start_run(run_name=f"model_{training_timestamp}"):

        mlflow.autolog()
        
        # Split data
        X = train_df[["AT", "V", "AP", "RH"]]
        y = train_df["PE"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Fit model
        model = RandomForestRegressor(max_depth=max_depth)
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        ## mlflow: log metrics
        mlflow.log_metrics({"testing_mse": mse, "testing_rmse": rmse})
        print(f"Test mse = {mse}, Test RMSE = {rmse}, Random forest max depth = {max_depth}")

In [41]:
for max_depth in range(2, 7, 2):
    _ = train_model(df, max_depth=max_depth)

2022/05/11 11:45:45 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/11 11:45:45 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/11 11:45:48 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/11 11:45:48 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Test mse = 37.429794758973244, Test RMSE = 6.117989437631716, Random forest max depth = 2


2022/05/11 11:45:53 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/11 11:45:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Test mse = 20.066638506544127, Test RMSE = 4.4795801707910226, Random forest max depth = 4
Test mse = 14.527805834285582, Test RMSE = 3.8115358891509317, Random forest max depth = 6


### Search runs

- [In the UI directly](https://www.mlflow.org/docs/latest/search-syntax.html#search)
- [Programmatically with search_runs](https://www.mlflow.org/docs/latest/search-syntax.html#programmatically-searching-runs)

- Get the id of the experiment where we want to search runs

In [42]:
mlflow.get_experiment_by_name(experiment_name)

<Experiment: artifact_location='/tmp/mlruns/1', experiment_id='1', lifecycle_stage='active', name='ep_prediction_with_random_forest', tags={}>

In [43]:
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
experiment_id

'1'

- Get all runs for the experiment

In [44]:
mlflow.search_runs(experiment_id)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_rmse,metrics.testing_mse,metrics.mean_squared_error_X_test,metrics.testing_rmse,metrics.training_score,metrics.training_r2_score,metrics.training_mse,metrics.mean_squared_error-2_X_test,metrics.training_mae,params.max_depth,params.bootstrap,params.min_impurity_decrease,params.warm_start,params.max_leaf_nodes,params.ccp_alpha,params.max_features,params.random_state,params.n_jobs,params.min_weight_fraction_leaf,params.oob_score,params.criterion,params.min_samples_split,params.min_samples_leaf,params.n_estimators,params.verbose,params.max_samples,tags.mlflow.source.type,tags.mlflow.user,tags.estimator_name,tags.mlflow.runName,tags.estimator_class,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.autologging
0,2d9576dcd51d4a748848cfce51706ad8,1,FINISHED,/tmp/mlruns/1/2d9576dcd51d4a748848cfce51706ad8...,2022-05-11 09:45:53.872000+00:00,2022-05-11 09:46:00.234000+00:00,3.820657,14.527806,14.527806,3.811536,0.949892,0.949892,14.597421,3.811536,2.917871,6,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,"model_2022-05-11, 11:45:53",sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""2d9576dcd51d4a748848cfce51706ad8""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...,None
1,67af7eeeca2a4181b18795a35669ba8e,1,FINISHED,/tmp/mlruns/1/67af7eeeca2a4181b18795a35669ba8e...,2022-05-11 09:45:48.840000+00:00,2022-05-11 09:45:53.864000+00:00,4.479851,20.066639,20.066639,4.479580,0.931110,0.931110,20.069066,4.479580,3.467432,4,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,"model_2022-05-11, 11:45:48",sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""67af7eeeca2a4181b18795a35669ba8e""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...,None
2,12503147cf1945be86a54e2aef89b1ef,1,FINISHED,/tmp/mlruns/1/12503147cf1945be86a54e2aef89b1ef...,2022-05-11 09:45:45.431000+00:00,2022-05-11 09:45:48.832000+00:00,6.096236,37.429795,37.429795,6.117989,0.872429,0.872429,37.164095,6.117989,4.816029,2,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,"model_2022-05-11, 11:45:45",sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""12503147cf1945be86a54e2aef89b1ef""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...,None
3,56d9ebc8f30848fab45e76b9af7bde77,1,FINISHED,/tmp/mlruns/1/56d9ebc8f30848fab45e76b9af7bde77...,2022-05-11 09:45:36.989000+00:00,2022-05-11 09:45:44.453000+00:00,3.827924,14.592045,14.592045,3.819954,0.949702,0.949702,14.653000,3.819954,2.924040,6,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,None,sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""56d9ebc8f30848fab45e76b9af7bde77""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...,None
4,bc85fbb304014881b8f84501f619a04a,1,FINISHED,/tmp/mlruns/1/bc85fbb304014881b8f84501f619a04a...,2022-05-11 09:45:30.912000+00:00,2022-05-11 09:45:36.981000+00:00,4.487612,20.130328,20.130328,4.486683,0.930871,0.930871,20.138663,4.486683,3.472453,4,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,None,sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""bc85fbb304014881b8f84501f619a04a""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...,None
5,6d6954a339cc4b2fa2dbdd1de3e22b78,1,FINISHED,/tmp/mlruns/1/6d6954a339cc4b2fa2dbdd1de3e22b78...,2022-05-11 09:45:26.381000+00:00,2022-05-11 09:45:30.886000+00:00,6.132042,37.887241,37.887241,6.155261,0.870926,0.870926,37.601945,6.155261,4.850076,2,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,None,sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""6d6954a339cc4b2fa2dbdd1de3e22b78""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...,None

- Filter runs by max_depth and mse and order them by mse

In [45]:
max_depth = 4
mlflow.search_runs(
    experiment_id,
    filter_string=f"params.max_depth = '{max_depth}' AND metrics.testing_mse <= 40",
    order_by=['metrics.testing_mse asc']
)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.testing_mse,metrics.testing_rmse,metrics.training_score,metrics.training_r2_score,metrics.training_mse,metrics.training_mae,metrics.training_rmse,metrics.mean_squared_error_X_test,metrics.mean_squared_error-2_X_test,params.max_depth,params.bootstrap,params.min_impurity_decrease,params.warm_start,params.max_leaf_nodes,params.ccp_alpha,params.max_features,params.random_state,params.n_jobs,params.min_weight_fraction_leaf,params.oob_score,params.criterion,params.min_samples_split,params.min_samples_leaf,params.n_estimators,params.verbose,params.max_samples,tags.mlflow.source.type,tags.mlflow.user,tags.estimator_name,tags.mlflow.runName,tags.estimator_class,tags.mlflow.log-model.history,tags.mlflow.source.name
0,2488a0995ab84da092d8a6c4c3a33e15,1,FINISHED,/tmp/mlruns/1/2488a0995ab84da092d8a6c4c3a33e15...,2022-05-11 09:37:07.459000+00:00,2022-05-11 09:37:12.778000+00:00,20.063128,4.479188,0.931098,0.931098,20.072466,3.465561,4.480231,NaN,NaN,4,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,"model_2022-05-11, 11:37:07",sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""2488a0995ab84da092d8a6c4c3a33e15""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...
1,67af7eeeca2a4181b18795a35669ba8e,1,FINISHED,/tmp/mlruns/1/67af7eeeca2a4181b18795a35669ba8e...,2022-05-11 09:45:48.840000+00:00,2022-05-11 09:45:53.864000+00:00,20.066639,4.479580,0.931110,0.931110,20.069066,3.467432,4.479851,20.066639,4.479580,4,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,"model_2022-05-11, 11:45:48",sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""67af7eeeca2a4181b18795a35669ba8e""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...
2,bc85fbb304014881b8f84501f619a04a,1,FINISHED,/tmp/mlruns/1/bc85fbb304014881b8f84501f619a04a...,2022-05-11 09:45:30.912000+00:00,2022-05-11 09:45:36.981000+00:00,20.130328,4.486683,0.930871,0.930871,20.138663,3.472453,4.487612,20.130328,4.486683,4,True,0.0,False,None,0.0,auto,None,None,0.0,False,squared_error,2,1,100,0,None,LOCAL,zx,RandomForestRegressor,None,sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""bc85fbb304014881b8f84501f619a04a""...",/Users/zx/miniconda3M1/envs/mlflow/lib/python3...


### Load a saved model

- [More informations on other format of model_uri](https://www.mlflow.org/docs/latest/python_api/mlflow.sklearn.html#mlflow.sklearn.load_model)

#### With the result of search_runs

In [46]:
run = mlflow.search_runs(
    experiment_id,
    filter_string=f"params.max_depth = '{max_depth}' AND metrics.testing_mse <= 30",
    order_by=["metrics.testing_mse asc"]
).iloc[0]
run

run_id                                                  2488a0995ab84da092d8a6c4c3a33e15
experiment_id                                                                          1
status                                                                          FINISHED
artifact_uri                           /tmp/mlruns/1/2488a0995ab84da092d8a6c4c3a33e15...
start_time                                              2022-05-11 09:37:07.459000+00:00
end_time                                                2022-05-11 09:37:12.778000+00:00
metrics.testing_mse                                                            20.063128
metrics.testing_rmse                                                            4.479188
metrics.training_score                                                          0.931098
metrics.training_r2_score                                                       0.931098
metrics.training_mse                                                           20.072466
metrics.training_mae 

In [47]:
run.artifact_uri

'/tmp/mlruns/1/2488a0995ab84da092d8a6c4c3a33e15/artifacts'

In [48]:
model = mlflow.sklearn.load_model(model_uri=f"{run.artifact_uri}/model")
model

RandomForestRegressor(max_depth=4)

In [49]:
model.predict(df[:5][["AT", "V", "AP", "RH"]])

array([464.47327488, 444.31862   , 485.83424254, 446.83289331,
       472.03424034])